In [ ]:
import ionerdss as ion
import os

pdb_id = '8y7s' # PDB ID for the structure of interest, or the full path to a PDB file
save_folder = f'{os.getcwd()}/{pdb_id}_dir' # the working directory

# create the PDBModel object using the PDBModel class
pdb_model = ion.PDBModel(pdb_id=pdb_id, save_dir=save_folder)

# coarse grain each chain of the PDB structure to a NERDSS molecule
# set standard_output=True to see the determined interfaces
pdb_model.coarse_grain(
    distance_cutoff=0.35,
    residue_cutoff=3,
    show_coarse_grained_structure=False,
    save_pymol_script=False,
    standard_output=False
    )

# regularize homologous chains to the same NERDSS molecule type
pdb_model.regularize_homologous_chains(
    dist_threshold_intra=3.5,
    dist_threshold_inter=3.5,
    angle_threshold=25.0,
    show_coarse_grained_structure=True,
    save_pymol_script=True,
    standard_output=False
    )

In [ ]:
from ionerdss import ParseComplexes
from ionerdss import ReactionStringParser
from ionerdss import solve_reaction_ode, reaction_dydt
import numpy as np

complex_list, complex_reaction_system = ParseComplexes(pdb_model)

# initialize an instance of reaction_string_parser
rsp = ReactionStringParser()

reaction_strings = [reaction.expression for reaction in complex_reaction_system.reactions]
species_names, rate_constant_names, reactant_matrix, product_matrix = rsp.parse_reaction_strings(reaction_strings)

# Rate constant assuming already non-dimensionalized
rate_constants = [reaction.rate for reaction in complex_reaction_system.reactions]

# Define time span and initial concentration, assuming already non-dimensionalized
t_span = [0.0, 10.0]
y_init = np.zeros(len(complex_list)) # initial concentration
y_init[0] = 1.0 # initial monomer concentration

time, concentrations, species_names = solve_reaction_ode(
    reaction_dydt, t_span,  y_init, reactant_matrix = reactant_matrix, product_matrix = product_matrix,
    k = rate_constants, plotting=False, method = "BDF", species_names = species_names
    )

import matplotlib.pyplot as plt

plt.plot(time,((concentrations.T)[0]).T, label='Monomer')
# Plotting the concentrations of Tetramer, sum of 1,2,3,4,5,6,7,8,9,10,12,13,14,15,16,17,24
indices = [1,2,3,4,5,6,7,8,9,10,12,13,14,15,16,17,24]
plt.plot(time,((concentrations.T)[indices]).T.sum(axis=1), label='Tetramer')
plt.xlabel('Time (s)')
plt.ylabel(r'Concentration $\left(\mu\mathrm{M}\right)$')
plt.legend()

In [ ]:
# create the Simulation object using the Simulation class
# the simulation is connected to the PDBModel object created above
simulation = ion.Simulation(pdb_model, save_folder)

# generate the NERDSS input files for the simulation
simulation.generate_nerdss_input()

simulation.modify_inp_file(
    {'nItr': 20000000, 'timeStep': 0.5,
    'timeWrite': 20000, 'trajWrite': 2000000,
    'pdbWrite': 2000000, 'A': 130,
    'WaterBox': [600.0, 600.0, 600.0]}
    )

simulation.print_inp_file()

In [ ]:
# uncomment next line install NERDSS if not already installed
simulation.install_nerdss(nerdss_path=save_folder)

In [ ]:
# run the NERDSS simulation
simulation.run_new_simulations(
    sim_indices=[1],
    sim_dir= save_folder + "/nerdss_output",
    nerdss_dir=save_folder + "/NERDSS",
    parallel=False
    )

In [ ]:
# create the Analysis object using the Analysis class
# the nerdss_output directory is the output directory from the NERDSS simulation
# it can be the parent directory of several simulations
analysis = ion.Analysis(save_folder + "/nerdss_output")

analysis.plot_figure(
    figure_type='line',
    x='time',
    y='count',
    legend=["A: 1.", "A: 4.",],
    show_type='average',
    figure_size = (6, 4)
    )

plt.plot(time,((concentrations.T)[0]).T * 130, label='Monomer (ODE)')
# Plotting the concentrations of Tetramer, sum of 1,2,3,4,5,6,7,8,9,10,12,13,14,15,16,17,24
indices = [1,2,3,4,5,6,7,8,9,10,12,13,14,15,16,17,24]
plt.plot(time,((concentrations.T)[indices]).T.sum(axis=1) * 130, label='Tetramer (ODE)')

plt.legend()